<a href="https://colab.research.google.com/github/sussygamedeveloper/eaglercraft-server-colab/blob/main/EaglerCraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Run a EaglerCraft Server on Google Colab!**

---

The script below will run your server. You'll have to create a server first to be able to use it - don't worry, the scripts below will do the majority of the work for you. You might also want to change the default region to your region, check below.

In [ ]:
import os
import re
import json

# Update the package lists
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"
# Install OpenJDK 17
# !wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
# !sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk17 gets installed."
!sudo apt-get install openjdk-8-jre-headless &>/dev/null && echo "Yay! Openjdk8 has been successfully installed." || echo "Failed to install OpenJdk17."
#Perform java version check
java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
if java_ver[0] == "8" :
  print("Openjdk8 is working correctly, you are good to go.")
else:
  print("Openjdk8 doesn't seems to be installed or isn't working, falling back to java", java_ver[0], "You Might Get Reduced Performance")

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Change directory to the Minecraft server folder on Google Drive
%cd "/content/drive/My Drive/Minecraft-server"
!ls #list the directory contents (to verify that working directory was changed)

# Import config file.
if os.path.isfile("colabconfig.json"):
  colabconfig = json.load(open("colabconfig.json"))
else:
  colabconfig = {"server_type": "generic"} # using default, if config doesn't exists.
  json.dump(colabconfig, open("colabconfig.json",'w'))

# Download Jar File
!wget https://raw.githubusercontent.com/LAX1DUDE/eaglercraft/main/stable-download/java/bukkit_command/craftbukkit-1.5.2-R1.0.jar

# Java arguments.
server_flags = "" # aiker's flags might negatively impact performance on non-paper servers.
memory_allocation = "-Xmx6144M -Xms6144M"

# Chose the tunnle service you want to use
# Available options: ngrok, argo
tunnel_service = "argo"
print("Procedding to use", tunnel_service)

if tunnel_service == "ngrok":
  !pip -q install pyngrok
  from pyngrok import conf, ngrok
  
  # Ask for the ngrok authtoken
  print("Get your authtoken from https://dashboard.ngrok.com/auth")
  import getpass
  authtoken = getpass.getpass()  # input your Ngrok auth token everytime you run the cell or simply replace "getpass.getpass()" with your token in "double quotes"
  ! ngrok authtoken $authtoken # login to ngrok

  # Sets default ngrok region
  conf.get_default().region = 'us'  # Change this to whichever region you want

  # Connect to ngrok
  url = ngrok.connect(25565, 'tcp')
  print('Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))

  print('Starting server...')
  !java $memory_allocation -jar craftbukkit-1.5.2-R1.0.jar nogui

else:
  # Download & make argo executable
  !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
  !chmod +x cloudflared-linux-amd64
  print('Starting server...')
  !./cloudflared-linux-amd64 tunnel --url tcp://127.0.0.1:25565 & java $memory_allocation -jar craftbukkit-1.5.2-R1.0.jar nogui


apt cache successfully updated
Yay! Openjdk8 has been successfully installed.
Openjdk8 doesn't seems to be installed or isn't working, falling back to java 11 You Might Get Reduced Performance
Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/My Drive/Minecraft-server'
/content
drive  sample_data
--2022-06-13 23:02:43--  https://raw.githubusercontent.com/LAX1DUDE/eaglercraft/main/stable-download/java/bukkit_command/craftbukkit-1.5.2-R1.0.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12893626 (12M) [application/octet-stream]
Saving to: ‘craftbukkit-1.5.2-R1.0.jar’

craftbukkit-1.5.2-R 100%[===================>]  12.30M  --.-KB/s    in 0.1s    

2022-06-13 23:02:45 (95.0 MB/s) - ‘craftbukkit-1.5.2-R1.0.jar’ saved [12893626/12

**Automatically accept the EULA**

In [15]:
# Please read the file stored in your server folder before running this command. 
# Also, go to https://www.minecraft.net/en-us/eula to read Minecraft's EULA.

# Make sure Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/Minecraft-server"
!echo "eula=true" >> eula.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/My Drive/Minecraft-server'
/content


In [ ]:
#Get public address (ngrok)
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"